In [1]:
import nltk 

with open('script.txt') as script:
    lines = script.readlines()

sentences = []

for line in lines:
    modified_line = ''
    for ch in line:
        ch = ch.lower()
        modified_line += ch
    # Remove unnecessary characters and whitespace
    modified_line = modified_line.replace('...', '').replace('\n', '').replace('!', '').replace('.', '').replace('\ufeff', '')
    if modified_line.strip():  # Check if the line is not empty after stripping whitespace
        sentences.append(modified_line)



from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser(url="http://localhost:9000")


CorenlpGrammarTree = next(parser.raw_parse(line))

import nltk

def tree_to_cfg(tree):
    cfg = {}

    def traverse(node):
        if isinstance(node, str):  # Terminal node
            return [node]
        else:
            children = [traverse(child) for child in node]
            non_terminal = node.label()
            if non_terminal not in cfg:
                cfg[non_terminal] = []
            cfg[non_terminal].extend(children)
            return [non_terminal]

    traverse(tree)
    return cfg

# Parse tree in the string format
parse_tree_str = str(CorenlpGrammarTree)

# Convert string to nltk.Tree object
parse_tree = nltk.Tree.fromstring(parse_tree_str)

# Convert the parse tree to CFG
cfg = tree_to_cfg(parse_tree)

# Print the CFG rules
for non_terminal, expansions in cfg.items():
    print(non_terminal + " -> " + " | ".join([" ".join(expansion) for expansion in expansions]))

DT -> the
JJ -> furry
NN -> lover
NP -> DT | JJ | NN
ROOT -> NP


In [4]:
import nltk
from nltk.parse.corenlp import CoreNLPParser

# Read script from file and preprocess sentences
sentences = []
with open('script.txt') as script:
    for line in script:
        modified_line = line.lower().strip().replace('...', '').replace('\n', '').replace('!', '').replace('.', '').replace('\ufeff', '')
        if modified_line:
            sentences.append(modified_line)

# Initialize CoreNLPParser
parser = CoreNLPParser(url="http://localhost:9000")

# Initialize an empty CFG
combined_cfg = {}

# Process each sentence and merge parse trees into a single CFG
for sentence in sentences:
    CorenlpGrammarTree = next(parser.raw_parse(sentence))

    def tree_to_cfg(tree):
        cfg = {}

        def traverse(node):
            if isinstance(node, str):  # Terminal node
                return [node]
            else:
                children = [traverse(child) for child in node]
                non_terminal = node.label()
                if non_terminal not in cfg:
                    cfg[non_terminal] = []
                cfg[non_terminal].extend(children)
                return [non_terminal]

        traverse(tree)
        return cfg

    # Convert the parse tree to CFG
    cfg = tree_to_cfg(CorenlpGrammarTree)

    # Merge the CFG of the current sentence into the combined CFG
    for non_terminal, expansions in cfg.items():
        if non_terminal not in combined_cfg:
            combined_cfg[non_terminal] = []
        combined_cfg[non_terminal].extend(expansions)

# Print the combined CFG rules
for non_terminal, expansions in combined_cfg.items():
    unique_expansions = set(tuple(expansion) for expansion in expansions)
    print(non_terminal + " -> " + " | ".join([" ".join(expansion) for expansion in unique_expansions]))

IN -> till | into | without | about | in | for | from | over | though | during | that | across | up | after | before | by | to | at | with | between | as | like | out | through | since | under | beyond | of | than | because | if | until | on
DT -> the | this | those | all | a | some | every | these | any | each | that | an | both | no
NNS -> games | clouds | ways | puss | cahoots | pieces | guards | dreams | moves | soldiers | backpacks | adiós | kids | eggs | gentlemen | orphans | plants | feet | paws | scars | sizes | months | lives | churches | thieves | friends | ambushes | goners | people | goggles | names | thanks | days | heels | rooms | adventures | claws | softpaws | ladies | outlaws | eyes | whiskers | socks | clocks | inglés | seconds | frijoles | boys | chances | beans | hands | lies | teenagers | lollipops | giants | guys | piggies | gracias | children | times | muffins | candlesticks | legends | chupacabra | feelings | onions | curtains | boots | cats | brothers | dukes |